In [50]:
import math
import hashlib
import inspect
import json
import pandas as pd
import numpy as np
import re
from hashlib import sha256
from collections import Counter
import string
import os
import random


import numpy as np
from numpy import dot
from numpy.linalg import norm


# from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import WordPunctTokenizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
import spacy
from spacy.matcher import Matcher
from nltk.tokenize import WordPunctTokenizer
from spacy.lang.en import English
from spacy.matcher import Matcher

from sklearn.feature_selection import SelectKBest, chi2

from sklearn.metrics import f1_score

from sklearn.metrics import confusion_matrix


In [51]:
df = pd.read_csv("hackathon/hckt04/HCKT04 - Text Classification/data/covid19_data.csv")

In [52]:
def true_or_false(x, next_phase=False): # clara's
    
    if next_phase:
        if x.lower() in ["correct", "true", "explanatory", "correct attribution", "news", "mostly true","half true", "collections", "half truth", "partially true"]:
            x = "true"
        else:
            x = "false"
    else:

        if x.lower() in ["correct", "true", "explanatory", "correct attribution", "news", "mostly true"]:
            x = "true"
        else:
            x = "false"
    return x

In [53]:
df["class"]=df["class"].apply(str).apply(true_or_false)

## Baseline

In [54]:
train_test_split?

In [55]:
# with just title, no preprocessing, vectorizer, chi2 selector, and random_forest clf

# Split in train and validation
train_data, test_data = train_test_split(df, test_size=0.2,stratify=df["class"])

# Build the pipeline
text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ("ch2" , SelectKBest(chi2, k=20)),
                     ('classifier', RandomForestClassifier())])

# Train the classifier
text_clf.fit(train_data['title'].values, train_data["class"].values)

predicted = text_clf.predict(test_data['title'].values)

f1_score(y_true=test_data["class"].values, y_pred=predicted, pos_label="true")

0.2608695652173913

In [56]:
confusion_matrix(test_data["class"].values, predicted)

array([[1087,    1],
       [  16,    3]])

In [57]:
def build_submission(predictions, file_name):
    submission = pd.DataFrame()
    submission['class'] = predictions
    submission.to_csv('./'+file_name+'.csv', index=False)
    return submission

In [58]:
# actual_test_df = pd.read_csv("data/covid19_unlabelled_test.csv")
# actual_pred = text_clf.predict(actual_test_df['title'].values)

# build_submission(actual_pred)

## Adding title length

In [59]:
df['title_length'] = df['title'].str.split().map(len)

In [60]:
class Selector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a column from the dataframe to perform additional transformations on
    """ 
    def __init__(self, key):
        self.key = key
        
    def fit(self, X, y=None):
        return self
    
    

class TextSelector(Selector):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on text columns in the data
    """
    def transform(self, X):
        return X[self.key]
    
    
class NumberSelector(Selector):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def transform(self, X):
        return X[[self.key]]

In [91]:
# Split in train and validation
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

# Build the pipeline
# first text
title_pipe = Pipeline([
                ('selector', TextSelector("title")),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2))),
                ("ch2" , SelectKBest(chi2, k=20))
                ])

# then numbers
title_lenght_pipe  = Pipeline([
                ('selector', NumberSelector("title_length")),
                ('scaler', StandardScaler())
                ])

# unite them

title_feats = FeatureUnion([('title', title_pipe), 
                            ('title_length', title_lenght_pipe)
                           ])



final_pipeline = Pipeline([
    ('title_feats',title_feats),
    ('classifier', RandomForestClassifier(random_state=42)),
])

count     5531
unique      96
top        AFP
freq       515
Name: verifiedby, dtype: object

In [83]:
final_pipeline.fit(train_data, train_data["class"].values)

predicted = final_pipeline.predict(test_data)

f1_score(y_true=test_data["class"].values, y_pred=predicted, pos_label="true")


0.25

In [74]:
(test_data["class"]=="true").sum()

13

In [75]:
confusion_matrix(test_data["class"], predicted)

array([[1094,    0],
       [   7,    6]])

In [76]:
actual_test_df = pd.read_csv("hackathon/hckt04/HCKT04 - Text Classification/data/covid19_unlabelled_test.csv")
actual_test_df['title_length'] = actual_test_df['title'].str.split().map(len)
actual_pred = final_pipeline.predict(actual_test_df)

build_submission(actual_pred, "submission2")

,class
0,false
1,true
2,false
3,false
4,false
...,...
2087,false
2088,false
2089,false
2090,false


## ver2.5 

In [108]:
# Split in train and validation
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

# Build the pipeline
# first title text
title_pipe = Pipeline([
                ('selector', TextSelector("title")),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2))),
                ("ch2" , SelectKBest(chi2, k=20))
                ])
#debunker titles translated
verified_pipe = Pipeline([
                ('selector', TextSelector("verifiedby")),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2))),
                ("ch2" , SelectKBest(chi2, k=20))
                ])

ref_source_pipe = Pipeline([
                ('selector', TextSelector("ref_source")),
                ('tfidf', TfidfVectorizer()),
                ("ch2" , SelectKBest(chi2, k=20))
                ])

#then numbers
title_lenght_pipe  = Pipeline([
                ('selector', NumberSelector("title_length")),
                ('scaler', StandardScaler())
                ])

# unite them

title_feats = FeatureUnion([
                            ('title', title_pipe),
                            ('verified_pipe', verified_pipe),
                            #('ref_source_pipe', ref_source_pipe),    
                            ('title_lenght_pipe', title_lenght_pipe) 
                           ])



final_pipeline = Pipeline([
    ('title_feats',title_feats),
    ('classifier', RandomForestClassifier(random_state=42)),
])


In [109]:
final_pipeline.fit(train_data, train_data["class"].values)

predicted = final_pipeline.predict(test_data)

f1_score(y_true=test_data["class"].values, y_pred=predicted, pos_label="true")


0.6666666666666667

In [110]:
(test_data["class"]=="true").sum()

13

In [111]:
confusion_matrix(test_data["class"], predicted)

array([[1093,    1],
       [   6,    7]])

In [112]:
actual_test_df = pd.read_csv("hackathon/hckt04/HCKT04 - Text Classification/data/covid19_unlabelled_test.csv")
actual_test_df['title_length'] = actual_test_df['title'].str.split().map(len)
actual_pred = final_pipeline.predict(actual_test_df)

build_submission(actual_pred, "submission2")

,class
0,false
1,true
2,false
3,false
4,false
...,...
2087,false
2088,false
2089,false
2090,false


## 3rd submission add country rate
